In [2]:
import json
import requests
from pprint import pprint

In [3]:
#Define the names for the data source, skillset, index and indexer
datasource_name="cogsrch-py-datasource"
skillset_name="cogsrch-py-skillset"
index_name="cogsrch-py-index"
indexer_name="cogsrch-py-indexer"

In [4]:
#Setup the endpoint
endpoint = 'https://hacker2cognitive.search.windows.net'
headers = {'Content-Type': 'application/json',
        'api-key': '151EF6A7E9AFC1914148E49C34C5CE65' }
params = {
    'api-version': '2019-05-06'
}

In [6]:
#Create a data source
datasourceConnectionString = 'DefaultEndpointsProtocol=https;AccountName=hacker2blob;AccountKey=V+K/nLb3K9pQzggDgrA2NEFQoxlz5ht9+f8q2Vz0RzaN8ppQ7vz9Mf8qXvAFEsbBxi1IleX7sWk6wGRaEZ32/w==;EndpointSuffix=core.windows.net'
#"<YOUR-BLOB-RESOURCE-CONNECTION-STRING>"
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
    "connectionString": datasourceConnectionString
   },
    "container": {
     "name": "challenge2"
   }
}
r = requests.put( endpoint + "/datasources/" + datasource_name, data=json.dumps(datasource_payload), headers=headers, params=params )
#pprint (r.json())

In [36]:
#Create a skillset
skillset_payload = {
  "name": skillset_name,
  "description":
  "Extract entities, detect language and entities and extract key-phrases",
  "skills":
  [
    # define entity recognition skill
    { 
      "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
      "categories": ["Location", "Person", "Datetime", "Organization"],
      "defaultLanguageCode": "en",
      "inputs": [
        {
          "name": "text", "source": "/document/content"
        }
      ],
      "outputs": [
        {
          "name": "locations", "targetName": "locations"
        },
        {
          "name": "persons", "targetName": "people"
        },
        {
          "name": "dateTimes", "targetName": "dateTimes"
        },
        {
          "name": "organizations", "targetName": "organizations" 
        }
      ]
    },
      
    #define language detection skill
    {
      "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
      "inputs": [
        {
          "name": "text", "source": "/document/content"
        }
      ],
      "outputs": [
        {
          "name": "languageCode"   # tempory field, only used by other skills. No need to write indexer
        }
      ]
    },
    
    # define sentiment scoring skill  
    {
    "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
    "inputs": [
        {
          "name": "text", "source": "/document/content"
        },
        {
          "name": "languageCode", "source": "/document/languageCode"
        }
    ],
    "outputs": [
        {
          "name": "score", "targetName": "mySentiment"
        }
    ]
    },
      
    # define text split skill
    {
      "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
      "textSplitMode" : "pages",
      "maximumPageLength": 4000,
      "inputs": [
        {
          "name": "text", "source": "/document/content"
        },
        {
          "name": "languageCode", "source": "/document/languageCode"
        }
      ],
      "outputs": [
        {
          "name": "textItems", "targetName": "pages"
        }
      ]
    },
      
    # define key phrase extraction skill
    {
      "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
      "context": "/document/pages/*",
      "inputs": [
        {
          "name": "text", "source": "/document/pages/*"
        },
        {
          "name":"languageCode", "source": "/document/languageCode"
        }
      ],
      "outputs": [
        {
          "name": "keyPhrases", "targetName": "keyPhrases"
        }
      ]
    },
      
    # define computer OCR skill and merge with source text
    {
      "description": "Extract text (plain and structured) from image.",
      "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
      "context": "/document/normalized_images/*",
      "defaultLanguageCode": "en",
      "detectOrientation": "true",
      "inputs": [
        {
          "name": "image",
          "source": "/document/normalized_images/*"
        }
      ],
      "outputs": [
        {
          "name": "text", "targetName" : "img_text"  
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
      "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field.",
      "context": "/document",
      "insertPreTag": " ",
      "insertPostTag": " ",
      "inputs": [
        {
          "name":"text", "source": "/document/content"
        },
        {
          "name": "itemsToInsert", "source": "/document/normalized_images/*/img_text"  # not text/*, one bounding box has one text
        },
        {
          "name":"offsets", "source": "/document/normalized_images/*/contentOffset" 
        }
      ],
      "outputs": [
        {
          "name": "mergedText", "targetName" : "merged_text"   
        }
      ]
    },
      
    # define image analysis skill
    {
      "description": "Extract image analysis.",
      "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
      "context": "/document/normalized_images/*",
      "defaultLanguageCode": "en",
      "visualFeatures": ["Tags", "Categories", "Description", "Faces"],
      "inputs": [
        {
          "name": "image",
          "source": "/document/normalized_images/*"
        }
      ],
      "outputs": [
        {
          "name": "categories"  # by default, target name is same as name
        },
        {
          "name": "tags"
        },
        {
          "name": "description"
        },
        {
          "name": "faces"
        }
      ]
    },
      
    # define customer skill. top 10 frequent words
    {"@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",  # not sure
        "description": "top 10 frenquent words custom skill",
        "uri": "https://kmcustom.azurewebsites.net/api/HttpTrigger1?code=XMcjDo0sExQ/9o/1e071Qr/yQaHTKNyM91Xo6FVY/6bhkalwoOD4Hw==",
        "batchSize":1,
        "context": "/document",
        "inputs": [
          {
            "name": "text", "source": "/document/content"
          },
          {
            "name": "language", "source": "/document/languageCode"
          }
        ],
        "outputs": [
          {
            "name": "text",
            "targetName": "topWord"  
          }
        ]
    }
  ],
  "cognitiveServices": 
  {
      "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
      "description": "mycogsvcs",
      "key": "4c1e9bb287a34e6aac498b5f97290ed7"
   }
}

r = requests.put(endpoint + "/skillsets/" + skillset_name, data=json.dumps(skillset_payload), headers=headers, params=params)
pprint(r.json())

{'@odata.context': 'https://hacker2cognitive.search.windows.net/$metadata#skillsets/$entity',
 '@odata.etag': '"0x8D6F023F55CFE57"',
 'cognitiveServices': {'@odata.type': '#Microsoft.Azure.Search.CognitiveServicesByKey',
                       'description': 'mycogsvcs',
                       'key': '4c1e9bb287a34e6aac498b5f97290ed7'},
 'description': 'Extract entities, detect language and entities and extract '
                'key-phrases',
 'name': 'cogsrch-py-skillset',
 'skills': [{'@odata.type': '#Microsoft.Skills.Text.EntityRecognitionSkill',
             'categories': ['Location', 'Person', 'Datetime', 'Organization'],
             'context': None,
             'defaultLanguageCode': 'en',
             'description': None,
             'includeTypelessEntities': None,
             'inputs': [{'inputs': [],
                         'name': 'text',
                         'source': '/document/content',
                         'sourceContext': None}],
             'minimumPreci

In [37]:
#Create an index
index_payload = {
    "name": index_name,
    "fields": [
      {
        "name": "id",
        "type": "Edm.String",
        "key": "true",
        "searchable": "true",
        "filterable": "false",
        "facetable": "false",
        "sortable": "true"
      },
      {
        "name": "url",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "file_name",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "content",
        "type": "Edm.String",
        "sortable": "false",
        "searchable": "true",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "size",
        "type": "Edm.Int64",
        "searchable": "false",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "last_modified",
        "type": "Edm.DateTimeOffset",
        "searchable": "false",
        "sortable": "false",
        "filterable": "true",
        "facetable": "false"
      },
      {
        "name": "sentiments",
        "type": "Edm.String",   # not sure
        "searchable": "true",
        "filterable": "false",
        "facetable": "false",
        "sortable": "true",
      },  
      {
        "name": "keyPhrases",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "locations",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "people",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "dateTimes",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "organizations",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
      },
      {
        "name": "imgText",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"   
      },
      {
        "name": "mergeText",
        "type": "Edm.String",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"   
      },
      {
        "name": "imgTags",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"   
      },
      {
        "name": "imgCategories",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"   
      },
      {
        "name": "imgDescription",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"   
      },
      {
        "name": "topWords",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"   
      }
   ]
}

r = requests.put(endpoint + "/indexes/" + index_name, data=json.dumps(index_payload), headers=headers, params=params)
pprint(r.json())

{'@odata.context': 'https://hacker2cognitive.search.windows.net/$metadata#indexes/$entity',
 '@odata.etag': '"0x8D6F023FB387B41"',
 'analyzers': [],
 'charFilters': [],
 'corsOptions': None,
 'defaultScoringProfile': None,
 'fields': [{'analyzer': None,
             'facetable': False,
             'filterable': False,
             'indexAnalyzer': None,
             'key': True,
             'name': 'id',
             'retrievable': True,
             'searchAnalyzer': None,
             'searchable': True,
             'sortable': True,
             'synonymMaps': [],
             'type': 'Edm.String'},
            {'analyzer': None,
             'facetable': False,
             'filterable': False,
             'indexAnalyzer': None,
             'key': False,
             'name': 'url',
             'retrievable': True,
             'searchAnalyzer': None,
             'searchable': False,
             'sortable': True,
             'synonymMaps': [],
             'type': 'Edm.Stri

In [ ]:
# Reset the indexer.

result = request.post(request_type="POST", function_name="/indexers/website-docs-indexer/reset", body=None)
if result != None:
    print(json.dumps(result, sort_keys=True, indent=2))

In [38]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "fieldMappings" : [
    {
      "sourceFieldName" : "metadata_storage_path",
      "targetFieldName" : "id",
      "mappingFunction" :
        { "name" : "base64Encode" }   # use mapping funtion for key (specify mapping method) 
    },
    {
      "sourceFieldName" : "metadata_storage_path",
      "targetFieldName" : "url"
    },
    {
      "sourceFieldName" : "metadata_storage_name",
      "targetFieldName" : "file_name"
    },
    {
      "sourceFieldName" : "metadata_content_encoding",
      "targetFieldName" : "content"
    },
    {
      "sourceFieldName" : "metadata_storage_size",
      "targetFieldName" : "size"
    },
    {
      "sourceFieldName" : "metadata_storage_last_modified",
      "targetFieldName" : "last_modified"
    }],
    "outputFieldMappings" :[ 
    {
      "sourceFieldName" : "/document/mySentiment",
      "targetFieldName" : "sentiments"
    },
    {
      "sourceFieldName" : "/document/locations/*",
      "targetFieldName" : "locations"
    },
    {
      "sourceFieldName" : "/document/people/*",
      "targetFieldName" : "people"
    },
    {
      "sourceFieldName" : "/document/dateTimes/*",
      "targetFieldName" : "dateTimes"
    },
    {
      "sourceFieldName" : "/document/organizations/*",
      "targetFieldName" : "organizations"
    },
    {
      "sourceFieldName" : "/document/pages/*/keyPhrases/*",
      "targetFieldName" : "keyPhrases"
    },
    {
      "sourceFieldName": "document/normalized_images/*/img_text",
      "targetFieldName" : "imgText"
    },
    {
      "sourceFieldName": "/document/merged_text", 
      "targetFieldName": "mergeText"   
    },
    {
      "sourceFieldName": "/document/normalized_images/*/tags/*",
      "targetFieldName": "imgTags" 
    },
    {
      "sourceFieldName": "/document/normalized_images/*/categories/*",
      "targetFieldName": "imgCategories" 
    },
    {
      "sourceFieldName": "/document/normalized_images/*/description",
      "targetFieldName": "imgDescription" 
    },
    {
      "sourceFieldName": "/document/topWord",
      "targetFieldName": "topWords"  
    }],
   "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration":
    {
      "dataToExtract": "contentAndMetadata",
      "imageAction": "generateNormalizedImages"
    }
  }
}

r = requests.put(endpoint + "/indexers/" + indexer_name, data=json.dumps(indexer_payload), headers=headers, params=params)
pprint(r.json())

{'@odata.context': 'https://hacker2cognitive.search.windows.net/$metadata#indexers/$entity',
 '@odata.etag': '"0x8D6F023FCFB3E92"',
 'dataSourceName': 'cogsrch-py-datasource',
 'description': None,
 'disabled': None,
 'fieldMappings': [{'mappingFunction': {'name': 'base64Encode',
                                        'parameters': None},
                    'sourceFieldName': 'metadata_storage_path',
                    'targetFieldName': 'id'},
                   {'mappingFunction': None,
                    'sourceFieldName': 'metadata_storage_path',
                    'targetFieldName': 'url'},
                   {'mappingFunction': None,
                    'sourceFieldName': 'metadata_storage_name',
                    'targetFieldName': 'file_name'},
                   {'mappingFunction': None,
                    'sourceFieldName': 'metadata_content_encoding',
                    'targetFieldName': 'content'},
                   {'mappingFunction': None,
                    '

In [39]:
# select the index to display

r = requests.get(endpoint + "/indexes/" + index_name + "/docs?&search=San Fransico&$select=file_name,organizations,topWords,imgCategories", headers=headers, params=params)

pprint(r.json())

{'@odata.context': "https://hacker2cognitive.search.windows.net/indexes('cogsrch-py-index')/$metadata#docs(*)",
 'value': [{'@search.score': 0.2897078,
            'file_name': 'San Francisco Brochure.pdf',
            'imgCategories': ['{"detail":{"celebrities":null,"landmarks":[{"name":"Golden '
                              'Gate '
                              'Bridge","confidence":0.96195101737976074}]},"name":"outdoor_","score":0.890625}',
                              '{"detail":{"celebrities":null,"landmarks":[]},"name":"building_street","score":0.9296875}'],
            'organizations': ['Travel Presents… \n'
                              '\n'
                              'San Francisco \n'
                              'San Francisco',
                              'City and County of San \n\nFrancisco',
                              'the United Nations',
                              'Los',
                              'OMB',
                              'The Wharf',
    

In [41]:
r = requests.get(endpoint + "/indexes/" + index_name + "/docs?&search=bridge\
                 &$select=file_name,keyPhrases,organizations,imgText&$searchFields=imgText", headers=headers, params=params) # specify search within image text

pprint(r.json())

{'@odata.context': "https://hacker2cognitive.search.windows.net/indexes('cogsrch-py-index')/$metadata#docs(*)",
 'value': [{'@search.score': 0.57381034,
            'file_name': 'San Francisco Brochure.pdf',
            'imgText': ['The Golden Gate Bridge', 'The Transamerica Pyramid'],
            'keyPhrases': ['San Francisco Peninsula',
                           'San Francisco Hotels',
                           'County of San',
                           'San Diego',
                           'OMB-designated San Jose-San Francisco-Oakland',
                           'Francisco Bay Area',
                           'populous city',
                           'large city',
                           'consolidated city-county',
                           'New York City',
                           'populated major city',
                           'United States',
                           'statistical area',
                           'Wharf Hotel',
                           'sta

In [53]:

#Query the index to return the contents of organizations
#Note: Index creation may take time. If this step returns no data, wait a few minutes
#      and then try again

params = dict(
    origin='Chicago,IL',
    destination='Los+Angeles,CA',
    waypoints='Joplin,MO|Oklahoma+City,OK',
    sensor='false'
)

r = requests.get(endpoint + "/indexes/" + index_name + "/docs?&search=London + Seattle&hightlight=description", headers=headers, params=params)
#print(r.json())
for value in r.json():
    print(value[0])
    print(value['content'])


@


TypeError: string indices must be integers

In [ ]:
import requests

url = 'http://maps.googleapis.com/maps/api/directions/json'

params = dict(
    origin='London',
    searchfield='person',
    waypoints='Joplin,MO|Oklahoma+City,OK',
    sensor='false'
)



#resp = requests.get(url=url, params=params)
#data = resp.json() 